# Three examples on using EFTfitter.jl

In [ ]:
using BAT, EFTfitter, Distributions, Plots, IntervalSets

Run the lines below if `Plotly` and/or `PyPlot` backends are not available (needed for some of the plots):

In [ ]:
# using Pkg
# Pkg.add("Plotly")
# ENV["PYTHON"]=""
# pkg"add PyPlot PyCall"

In [ ]:
#using Pkg
#pkg"add BAT, EFTfitter"

## Example 1: BLUE-like combination of multiple measurements of the same observable

Multiple **correlated** measurements of the charm quark lifetime (all in units of $10^{-13}$ seconds):

$\tau_1 =  9.5 \pm 1.66$   
$\tau_2 = 11.9 \pm 1.29$  
$\tau_3 = 11.1 \pm 1.45$   
$\tau_4 =  8.9 \pm 1.71$  
 
Goal: Determine the best combined value, i.e.,  BLUE - Best Linear Unbiased Estimator   
(Example based on http://cds.cern.ch/record/183996/files/OUNP-88-05.pdf?subformat=pdfa&version=1)

### Define a parameter for estimating the combined value:

In [ ]:
parameters = BAT.NamedTupleDist(
    τ = 7..15,  # uniform prior distribution between 7 and 15
);


#no underlying model -> observable == parameter:
estimator(params) = params.τ 

### Insert the measurements & uncertainties:

In [ ]:
measurements = (
    τ1 = Measurement(estimator,  9.50, uncertainties = (exp=1.66,) ),
    τ2 = Measurement(estimator, 11.90, uncertainties = (exp=1.29,) ),
    τ3 = Measurement(estimator, 11.10, uncertainties = (exp=1.45,) ),
    τ4 = Measurement(estimator,  8.90, uncertainties = (exp=1.71,) ),
)

### Insert the correlations of the uncertainties:

In [ ]:
correlations = (
    exp = Correlation([1.0 0.54 0.36 0.46;
                        0.54 1.0 0.44 0.60;
                        0.36 0.44 1.0 0.42;
                        0.46 0.60 0.42 1.0]),
);

### Build the model & posterior distribution:

In [ ]:
model = EFTfitterModel(parameters, measurements, correlations)
posterior = PosteriorDensity(model);

### Choose algorithm & sample the posterior: 

In [ ]:
algorithm = MCMCSampling(mcalg = MetropolisHastings(), nsteps = 10^6, nchains = 4)
samples = bat_sample(posterior, algorithm).result;

### Plot the posterior:

In [ ]:
Plots.plot(samples, :τ)

### Print numerical results:

In [ ]:
println("Mode: $(mode(samples).τ[1])")
println("Mean: $(mean(samples).τ[1]) ± $(std(samples).τ[1])", "\n")

### Comparison with classical BLUE result:

In [ ]:
blue = BLUE(model)

println("BLUE: $(blue.value) ± $(blue.unc)")

## Example 2: Combining different observables for constraining underlying parameters

In this example, measurements of two different observables are combined to constrain underlying parameters.

Specifically, we want to determine the gravitational acceleration $g$ using a measurement of a free fall distance and a measurement of a pendulum period.

We assume the following **observables**:

Free fall distance: $S = \frac{1}{2} \cdot g \cdot t^2$

Pendulum period: $T = 2\pi \sqrt{\frac{L}{g}}$, where $L$ is the length of the pendulum

And the corresponding **measurements** are assumed to be:  

$S = (1102.6 \pm  55.2)$ m

$T = (4.5 \pm 0.2)$ s

### Define parameters:

In [ ]:
parameters = BAT.NamedTupleDist(
    g  = 0.1..20,                            # gravitational acceleration to be constrained (uniform prior)
    t  = Normal(15., 0.5),                   # time of free fall  (known with some uncertainty)
    L  = truncated(Normal(5., 0.5), 0.1, 10) # length of pendulum (known with some uncertainty)
);


### Implement dependeces of the observables on the parameters:

In [ ]:
function distance(params)
    return 0.5*params.g*(params.t^2)
end

function period(params)
    return 2π*sqrt(params.L/params.g)
end

### Insert measurements, uncertainties and correlations:

In [ ]:
measurements = (
    distance_meas = Measurement(distance, 1102.6, uncertainties = (exp=55.2,), active = true),
    period_meas   = Measurement(period,      4.5, uncertainties = (exp=0.2,), active = true),
)


correlations = (
    exp = NoCorrelation(),
);

### Build the model & posterior:

In [ ]:
model = EFTfitterModel(parameters, measurements, correlations)
posterior = PosteriorDensity(model);

### Define algorithm & generate samples:

In [ ]:
algorithm = MCMCSampling(mcalg = MetropolisHastings(), nsteps = 10^6, nchains = 4)
samples = bat_sample(posterior, algorithm).result;

In [ ]:
plot(samples)

# plot prior distributions ontop to visualize gain of knowledge:
# plot!(parameters)

### Print mean values:

In [ ]:
println("Mean(g) = $(mean(samples).g[1]) ± $(std(samples).g[1])", "\n")
println("Mean(t) = $(mean(samples).t[1]) ± $(std(samples).t[1])", "\n")
println("Mean(L) = $(mean(samples).L[1]) ± $(std(samples).L[1])", "\n")

#     
      

## Example 3: EFT fit to top-quark & B measurements

This example is a SMEFT fit of the three Wilson coefficients $C_{uB}$, $C_{uG}$, and $C_{uW}$ to:   
- a $B$ physics measurement (branching ratio of $\bar B \rightarrow X_s \gamma$)   
- and a top-quark measurement (fiducial cross section of $t\bar t\gamma$ production).

(Example based on https://inspirehep.net/files/7fd8cefc6c3ba009e77eee7d8fab6734)

###  Define parameters and priors:

In [ ]:
parameters = BAT.NamedTupleDist(
    CuB  = -1..1, # uniform prior distributions for all Wilson coefficients
    CuG  = -1..1,
    CuW  = -1..1,
);

### Define the depences of the observables on the parameters:

The depence of the observables on the Wilson coefficients is determined outside EFTfitter using theory calculations and MC simulations.

### Branching ratio:

In [ ]:
function BR_Xsgamma(params)
    C = B_coeff(params)
    p = params_BR_BXsgamma()
    return compute_BR(C, p)
end



#=== The functions below contain everything needed for calculating the BR as a function of the Wilson coefficients ===#
function params_BR_BXsgamma()
    return [ 0.000336098480234567,
     -0.0009933924103794787, -0.00020718442307888992, 0.0, 0.0,
     0.0008165776679190035, 0.00031795137986217944, 1.6263032587282567e-19,
     1.6263032587282567e-19, 4.987993887589637e-05, -5.421010862427522e-20,
     -5.421010862427522e-20, 0.0, 0.0, 0.0
     ]
end

function compute_BR(C, p)
    BR = ( p[1] + p[2]*C[1] + p[3]*C[2] + p[4]*C[3]
    + p[5]*C[4] + p[6]*C[1]*C[1] + p[7]*C[1]*C[2]
    + p[8]*C[1]*C[3] + p[9]*C[1]*C[4] + p[10]*C[2]*C[2]
    + p[11]*C[2]*C[3] + p[12]*C[2]*C[4] + p[13]*C[3]*C[3]
    + p[14]*C[3]*C[4] + p[15]*C[4]*C[4] )

    return BR
end

match_C7(C) = -2.400 * C[1] + 0.09581 * C[3] 
match_C8(C) = -0.6821 * C[2] + 0.2519 * C[3] 
match_C9(C) =  2.440 * C[1] + 2.16032 * C[3]
match_C10(C) = 7.650 * C[3]
match_C1(C) = 4.402 * C[3]
match_CL(C) = -2.8233 * C[3]

function B_coeff(params)
    C = run_SMEFT(params)

    C7  = match_C7(C)
    C8  = match_C8(C)
    C9  = match_C9(C)
    C10 = match_C10(C)

    return [C7, C8, C9, C10]
end


function run_SMEFT(params)
    cub = params.CuB[1]
    cug = params.CuG[1] 
    cuw = params.CuW[1]

    coeff = [ cub, cug, cuw]

    coeff[1] = 0.949 * cub - 0.009 * cug + 0.0007 * cuw
    coeff[2] = -0.0068 * cub + 1.011 * cug - 0.023 * cuw
    coeff[3] = 0.00023 * cub - 0.009 * cug + 0.96 * cuw

    return coeff
end

### fiducial cross section:

In [ ]:
function tta_13_1l(params)
    cub = params.CuB
    cug = params.CuG
    cuw = params.CuW

    θw = 28.13
    sw = sind(θw)
    cw = cosd(θw)
    cuz = cw*cuw - sw*cub
    fiducial_xsec_1l_uz([cug, cuw, cuz])
end


#=== The functions below contain everything needed for calculating the fiducial cross section as a function of the Wilson coefficients ===#
function fiducial_xsec_1l_uz(C)
    k_1l = 83.4665999459313
    fit_eff_param_1l =  params_fidacc_1l()
    interpol_noacc_param = params_total_xsec()

    return eff(C, fit_eff_param_1l) .* quadratic_model(C, interpol_noacc_param) .+ k_1l
end

# fiducial acceptance
function eff(C, e)
    s = params_total_xsec()
    denominator = quadratic_model(C, s)

    numerator = (e[1]*s[1]*C[1]^0*C[2]^0*C[3]^0
    + e[2]*s[2]*C[1]^0*C[2]^0*C[3]^1
    + e[3]*s[3]*C[1]^0*C[2]^0*C[3]^2
    + e[4]*s[4]*C[1]^0*C[2]^1*C[3]^0
    + e[5]*s[5]*C[1]^0*C[2]^1*C[3]^1
    + e[6]*s[6]*C[1]^0*C[2]^2*C[3]^0
    + e[7]*s[7]*C[1]^1*C[2]^0*C[3]^0
    + e[8]*s[8]*C[1]^1*C[2]^0*C[3]^1
    + e[9]*s[9]*C[1]^1*C[2]^1*C[3]^0
    + e[10]*s[10]*C[1]^2*C[2]^0*C[3]^0
    )

    return numerator ./ denominator
end

function quadratic_model(C, p)
(p[1]*C[1]^0*C[2]^0*C[3]^0
+ p[2]*C[1]^0*C[2]^0*C[3]^1
+ p[3]*C[1]^0*C[2]^0*C[3]^2
+ p[4]*C[1]^0*C[2]^1*C[3]^0
+ p[5]*C[1]^0*C[2]^1*C[3]^1
+ p[6]*C[1]^0*C[2]^2*C[3]^0
+ p[7]*C[1]^1*C[2]^0*C[3]^0
+ p[8]*C[1]^1*C[2]^0*C[3]^1
+ p[9]*C[1]^1*C[2]^1*C[3]^0
+ p[10]*C[1]^2*C[2]^0*C[3]^0
)
end

function params_total_xsec()
    return [4787.8319388679765,
    -633.3295852678285, 44776.40427336053, 26525.758407582485,
    -102898.33073686132, 105571.83244747801, 20441.62967975978,
    -8672.01708116833, 64965.31092132101, 51975.059043423]
end

function params_fidacc_1l()
    return [0.08595401954550866,
    0.24342261870353674, 0.23993945397597446, 0.08809735354860272,
    0.24064846695181866, 0.17129137428997598, 0.08453761417847493,
    0.2571572039502875, 0.11016589914607273, 0.10102226849696228]
end

### Insert the measurements and uncertainties:

In [ ]:
measurements = (
    HFLAV_BR_Xsgamma  = Measurement(BR_Xsgamma, 332e-6, uncertainties = (exp=15e-6, theo=23e-6), active = true),
    ATLAS_13_1l       = Measurement(tta_13_1l,  521.0,  uncertainties = (exp =42. , theo=99.),   active = true),
);

### and (no) correlations:  

In [ ]:
correlations = (
    exp  = NoCorrelation(),
    theo = NoCorrelation()
);

### Build the Model:

In [ ]:
m = EFTfitterModel(parameters, measurements, correlations);

### Inspect the models & measurements:

In [ ]:
p = plot( get_observables(m).tta_13_1l, (CuB=-1:0.01:1, CuG=0, CuW=0), label="CuB", lw=1.5)
p = plot!(get_observables(m).tta_13_1l, (CuB=0, CuG=-1:0.01:1, CuW=0), label="CuG", lw=1.5)
p = plot!(get_observables(m).tta_13_1l, (CuB=0, CuG=0, CuW=-1:0.01:1), label="CuW", lw=1.5)

p = plot!(get_measurements(m).ATLAS_13_1l, label="ATLAS Measurement", color=:steelblue2)

p = plot!(xlabel="C", title="ttbar gamma cross section", legend = :topleft,)

In [ ]:
p = plot( get_observables(m).BR_Xsgamma, (CuB=-1:0.01:1, CuG=0, CuW=0), label="CuB", lw=1.5)
p = plot!(get_observables(m).BR_Xsgamma, (CuB=0, CuG=-1:0.01:1, CuW=0), label="CuG", lw=1.5)
p = plot!(get_observables(m).BR_Xsgamma, (CuB=0, CuG=0, CuW=-1:0.01:1), label="CuW", lw=1.5)

p = plot!(get_measurements(m).HFLAV_BR_Xsgamma, label="HFLAV measurement", color=:grey56)

p = plot!(xlabel="C", title="Branching ratio", legend = :topleft, ylims=(0, 0.001))

### Sample the posterior distribution of the combination:

In [ ]:
posterior = PosteriorDensity(m)
algorithm = MCMCSampling(mcalg = MetropolisHastings(), nsteps = 10^6, nchains = 4)
samples = bat_sample(posterior, algorithm).result;

### Plot samples of combination:

In [ ]:
plot(samples)

### More infos on plotting with BAT.jl and EFTfitter.jl:

https://bat.github.io/BAT.jl/dev/plotting/

https://tudo-physik-e4.github.io/EFTfitter.jl/dev/plotting/  
  
    
    

### Fit using only B the measurement:

In [ ]:
measurements_B = (
    HFLAV_BR_Xsgamma  = Measurement(BR_Xsgamma , 332e-6, uncertainties = (exp=15e-6, theo=23e-6), active = true),
    ATLAS_13_1l       = Measurement(tta_13_1l, 521.0, uncertainties = (exp =42. , theo=99.), active = false),
);

m_B = EFTfitterModel(parameters, measurements_B, correlations);
posterior_B = PosteriorDensity(m_B)
samples_B = bat_sample(posterior_B, algorithm).result;

### Fit using only the top-quark measurement:

In [ ]:
measurements_top = (
    HFLAV_BR_Xsgamma  = Measurement(BR_Xsgamma , 332e-6, uncertainties = (exp=15e-6, theo=23e-6), active = false),
    ATLAS_13_1l       = Measurement(tta_13_1l, 521.0, uncertainties = (exp =42. , theo=99.), active = true),
);

m_top = EFTfitterModel(parameters, measurements_top, correlations);
posterior_top = PosteriorDensity(m_top)
samples_top = bat_sample(posterior_top, algorithm).result;

### 2D Plots of the 90% areas:

In [ ]:
f = 1.6
pyplot(size=(f*400, f*300))

Ci = :CuB ; Cj = :CuW

p = plot( samples_B,   (Ci, Cj), intervals=[0.9], colors=[:gray86],     lw=1, st=:smallest_intervals_contourf, alpha=0.5, bins=80, smoothing=0.5, marginalmode=false, label="only B")
p = plot!(samples_top, (Ci, Cj), intervals=[0.9], colors=[:steelblue2], lw=1, st=:smallest_intervals_contourf, alpha=0.5, bins=80, smoothing=0.8, marginalmode=false)
p = plot!(samples,     (Ci, Cj), intervals=[0.9], colors=[:darkblue],   lw=1, st=:smallest_intervals_contourf, alpha=0.5, bins=80, smoothing=1.0, marginalmode=false)


p = plot!(xlims=(-1, 1), ylims=(-1, 1))

### 3D Plots

In [ ]:
N = 40000

comb_x = [BAT.unshaped.(samples).v[i][1] for i in 1:length(samples)][end-N:end]
comb_y = [BAT.unshaped.(samples).v[i][2] for i in 1:length(samples)][end-N:end]
comb_z = [BAT.unshaped.(samples).v[i][3] for i in 1:length(samples)][end-N:end]

B_x = [BAT.unshaped.(samples_B).v[i][1] for i in 1:length(samples_B)][end-2N:end]
B_y = [BAT.unshaped.(samples_B).v[i][2] for i in 1:length(samples_B)][end-2N:end]
B_z = [BAT.unshaped.(samples_B).v[i][3] for i in 1:length(samples_B)][end-2N:end]

top_x = [BAT.unshaped.(samples_top).v[i][1] for i in 1:length(samples_top)][end-2N:end]
top_y = [BAT.unshaped.(samples_top).v[i][2] for i in 1:length(samples_top)][end-2N:end]
top_z = [BAT.unshaped.(samples_top).v[i][3] for i in 1:length(samples_top)][end-2N:end]



f = 2
plotly(size=(f*400, f*300))

ms = 0.7 # markersize
p = Plots.scatter(B_x, B_y, B_z, ms=ms, alpha=1, color=:gray72, msw=0, xlabel="CuB", ylabel="CuG", zlabel="CuW", label="only B")
p = Plots.scatter!(top_x, top_y, top_z, ms=ms, color=:steelblue2, msw=0, label="only top", alpha=1)
p = Plots.scatter!(comb_x, comb_y, comb_z, ms=ms, color=:darkblue, msw=0, label="combination", )

### Plot 1D 90% intervals: 

In [ ]:
f = 1.5
pyplot(size=(f*400, f*300))

plot( samples_B,   0.9, y_offset=-0.2, atol=0.1, msw=4, ms=8, msc=:gray56,     label="only B")
plot!(samples_top, 0.9,                atol=0.1, msw=4, ms=8, msc=:steelblue2, label="only Top")
plot!(samples,     0.9, y_offset=0.2,  atol=0.1, msw=4, ms=8, msc=:darkblue,   label="combination")

plot!(title="90% intervals", legend=true)